In [1]:
from transformers import AutoTokenizer, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")  

inputs = tokenizer(["a photo of a cat", "a photo of a dog", "a photo of a doctor"], padding=True, return_tensors="pt")
text_features = model.get_text_features(**inputs)

c:\Users\ra78lof\AppData\Local\anaconda3\envs\ucd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# debug

print(text_features.shape)

torch.Size([3, 512])


In [3]:
import torch

In [4]:
Numerator = text_features@text_features.transpose(1, 0)
print(Numerator.shape)  
print(Numerator)


torch.Size([3, 3])
tensor([[109.7550, 104.3795, 100.9567],
        [104.3795, 114.5316, 105.4316],
        [100.9567, 105.4316, 122.9855]], grad_fn=<MmBackward0>)


In [5]:
#print(text_features.norm(dim=1).shape) 
#print(text_features.norm(dim=1).unsqueeze(0).shape)
#print(text_features.norm(dim=1).unsqueeze(1).shape)    

In [6]:
Denomitor = text_features.norm(dim=1).unsqueeze(1)@ text_features.norm(dim=1).unsqueeze(0)
print(Denomitor.shape)  
print(Denomitor)

torch.Size([3, 3])
tensor([[109.7550, 112.1179, 116.1821],
        [112.1179, 114.5316, 118.6833],
        [116.1821, 118.6833, 122.9855]], grad_fn=<MmBackward0>)


In [8]:
clip_concept_similarity = Numerator/Denomitor 

print(clip_concept_similarity)

tensor([[1.0000, 0.9310, 0.8690],
        [0.9310, 1.0000, 0.8883],
        [0.8690, 0.8883, 1.0000]], grad_fn=<DivBackward0>)


In [9]:
compare_input = tokenizer(['a photo of a tiger'], padding=True, return_tensors="pt")
compare_text_features = model.get_text_features(**compare_input)
print(compare_text_features.shape)

torch.Size([1, 512])


In [11]:
# compare the similarity between the compare input and the text features
cosine_similarity = torch.cosine_similarity(text_features, compare_text_features, axis=1)
print(cosine_similarity.shape)
print(cosine_similarity)

torch.Size([3])
tensor([0.8940, 0.8714, 0.8360], grad_fn=<SumBackward1>)


In [12]:
import numpy as np

In [15]:
similar_words = (np.array(cosine_similarity.detach().cpu()) > 0.85).nonzero()[0]
print(similar_words)

[0 1]
